In [3]:
import pandas as pd

dataset_path = r"C:\Users\a0494\Desktop\Diploma\data\russian_propaganda_tweets\russian_propaganda_tweets_v2.csv"

df = pd.read_csv(dataset_path)
print(df.head())
dataset_path = r"C:\Users\a0494\Desktop\Diploma\data\russian_propaganda_tweets\western_analysts_tweets.csv"

df1 = pd.read_csv(dataset_path, encoding='latin_1')
print(df1.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_prop = df[columns_to_keep]
#print(df_prop.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_western = df1[columns_to_keep]
df_western.head()


import pandas as pd

df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1

df_merged = pd.concat([df_prop, df_western])  # Об'єднуємо два датасети
df_merged = df_merged[df_merged['language'] == 'en']

          id  conversation_id                             created_at  \
0  1500000.0     1.500000e+18  2022-02-24 14:01:58 GTB Standard Time   
1  1500000.0     1.500000e+18  2022-02-24 14:24:32 GTB Standard Time   
2  1500000.0     1.500000e+18  2022-02-24 14:42:53 GTB Standard Time   
3  1500000.0     1.500000e+18  2022-02-24 14:50:06 GTB Standard Time   
4  1500000.0     1.500000e+18  2022-02-24 14:56:54 GTB Standard Time   

         dat      time  timezone      user_id        username  \
0  2/24/2022  14:01:58       300  344563802.0        tinkzorg   
1  2/24/2022  14:24:32       300  255471924.0      mfa_russia   
2  2/24/2022  14:42:53       300  255471924.0      mfa_russia   
3  2/24/2022  14:50:06       300  114718372.0  russianembassy   
4  2/24/2022  14:56:54       300  255471924.0      mfa_russia   

                            name  place  ... geo source user_rt_id user_rt  \
0  Morgenthau Plan Respecter 40K    NaN  ... NaN    NaN        NaN     NaN   
1                  M

C:\Users\a0494\AppData\Local\Temp\ipykernel_21364\161055038.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
C:\Users\a0494\AppData\Local\Temp\ipykernel_21364\161055038.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1


In [4]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

# Assuming you have a train dataset, you can replace it with your actual dataset
columns_to_keep = ['tweet', 'dataset']
df_merged_cropped = df_merged[columns_to_keep]
train_dataset, test_dataset = train_test_split(df_merged_cropped, test_size=0.2, random_state=42)

# Word count filter
train_dataset['word_count'] = train_dataset['tweet'].apply(lambda x: len(str(x).split()))
train_dataset = train_dataset[train_dataset['word_count'] <= 30]
train_dataset = train_dataset.drop(columns=['word_count'])

# Word count filter for test dataset
test_dataset['word_count'] = test_dataset['tweet'].apply(lambda x: len(str(x).split()))
test_dataset = test_dataset[test_dataset['word_count'] <= 30]
test_dataset = test_dataset.drop(columns=['word_count'])
# TextVectorization
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(np.array(train_dataset['tweet']))

# Save feature names
feature_names = encoder.get_vocabulary()

In [24]:

from keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization

import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
#from keras.layers.experimental.preprocessing import TextVectorization
# Model
model = Sequential([
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=128, mask_zero=True),
    GlobalAveragePooling1D(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
X_train = np.array(train_dataset['tweet'])
y_train = np.array(train_dataset['dataset'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Evaluate on test dataset
X_test = np.array(test_dataset['tweet'])
y_test = np.array(test_dataset['dataset'])
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
454/454 [==============================] - 10s 18ms/step - loss: 0.5173 - accuracy: 0.7618 - val_loss: 0.4329 - val_accuracy: 0.8039
Epoch 2/5
454/454 [==============================] - 8s 18ms/step - loss: 0.3308 - accuracy: 0.8640 - val_loss: 0.3703 - val_accuracy: 0.8394
Epoch 3/5
454/454 [==============================] - 8s 18ms/step - loss: 0.2425 - accuracy: 0.9059 - val_loss: 0.3529 - val_accuracy: 0.8534
Epoch 4/5
454/454 [==============================] - 8s 17ms/step - loss: 0.1889 - accuracy: 0.9295 - val_loss: 0.3548 - val_accuracy: 0.8567
Epoch 5/5
146/146 [==============================] - 1s 4ms/step - loss: 0.3832 - accuracy: 0.8495
Test accuracy: 0.8494969010353088


In [7]:
from keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
#from keras.layers.experimental.preprocessing import TextVectorization
# Model
model = Sequential([
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=256, mask_zero=True),
    Dropout(0.5),  # Доданий dropout для регуляризації
    BatchNormalization(),  # Доданий BatchNormalization для стабільності та швидкості навчання
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),  # Доданий Dense-шар з функцією активації ReLU
    Dropout(0.5),  # Доданий dropout для регуляризації
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
X_train = np.array(train_dataset['tweet'])
y_train = np.array(train_dataset['dataset'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Evaluate on test dataset
X_test = np.array(test_dataset['tweet'])
y_test = np.array(test_dataset['dataset'])
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
454/454 [==============================] - 14s 29ms/step - loss: 0.4607 - accuracy: 0.7827 - val_loss: 0.4207 - val_accuracy: 0.8069
Epoch 2/5
454/454 [==============================] - 13s 28ms/step - loss: 0.3013 - accuracy: 0.8737 - val_loss: 0.3594 - val_accuracy: 0.8452
Epoch 3/5
454/454 [==============================] - 13s 28ms/step - loss: 0.2215 - accuracy: 0.9105 - val_loss: 0.3884 - val_accuracy: 0.8493
Epoch 4/5
454/454 [==============================] - 14s 30ms/step - loss: 0.1817 - accuracy: 0.9262 - val_loss: 0.4162 - val_accuracy: 0.8457
Epoch 5/5
146/146 [==============================] - 0s 2ms/step - loss: 0.4970 - accuracy: 0.8401
Test accuracy: 0.8400770425796509
